In [ ]:
from pyoxigraph import Store, NamedNode, Literal, Variable, serialize, RdfFormat

#### Fedora 4 objects

In [ ]:
store = Store.read_only('./db')

In [ ]:
# Looking for null values in the proxyFor relation
query_null = '''
PREFIX ns001: <info:fedora/fedora-system:def/model#>
PREFIX ns012: <http://www.openarchives.org/ore/terms/>

SELECT DISTINCT ?s
WHERE 
{
    ?s ns001:hasModel ?o1;
         ns012:proxyFor ?o2.
    FILTER (?o1 = "ActiveFedora::Aggregation::Proxy")
    FILTER (STRLEN(?o2) = 0) 
} 
'''

In [ ]:
# Find all unique values of the proxyFor relation
query = '''
PREFIX ns001: <info:fedora/fedora-system:def/model#>
PREFIX ns012: <http://www.openarchives.org/ore/terms/>

SELECT DISTINCT ?o2
WHERE 
{
    ?s ns001:hasModel ?o1;
         ns012:proxyFor ?o2.
    FILTER (?o1 = "ActiveFedora::Aggregation::Proxy")
} 
'''

In [ ]:
results = store.query(query)

In [ ]:
results = list(results)

In [ ]:
[r for r in results if not isinstance(r[Variable('o2')], NamedNode)]

In [ ]:
# Find any objects of the proxyFor relation that are not subjects in the graph
query_orphan = '''
PREFIX ns001: <info:fedora/fedora-system:def/model#>
PREFIX ns012: <http://www.openarchives.org/ore/terms/>

SELECT DISTINCT ?o2
WHERE 
{
    ?s ns001:hasModel ?o1;
         ns012:proxyFor ?o2.
    FILTER (?o1 = "ActiveFedora::Aggregation::Proxy")
    FILTER NOT EXISTS { 
        ?o2 ?p ?o3 .
    }
} 
'''

In [ ]:
results = store.query(query)

In [ ]:
len(list(results))

In [ ]:
# Find proxy containers lacking the proxyFor predicate
query_dangling = '''
PREFIX ns001: <info:fedora/fedora-system:def/model#>
PREFIX ns012: <http://www.openarchives.org/ore/terms/>

SELECT DISTINCT ?s
WHERE 
{
    ?s ns001:hasModel ?o1;
    FILTER (?o1 = "ActiveFedora::Aggregation::Proxy")
    FILTER NOT EXISTS { 
        ?s ns012:proxyFor ?o2.
    }
} 
'''

In [ ]:
results = list(store.query(query_dangling))
len(results)

In [ ]:
with open('dangling-objects.txt', 'w') as f:
    for result in results:
        row = result['s'].value
        f.write(f'{row}\n')

In [ ]:
results[0]

This record, for instance

- `hasModel` = `ActiveFedora::Aggregation::Proxy`
- `proxyFor` **is missing**
- `hasParent` = `http://localhost:8984/rest/prod/fx/71/9n/26/fx719n26s/members`, which `hasModel` = `ActiveFedora::IndirectContainer`
    - the parent `hasParent` = `http://localhost:8984/rest/prod/fx/71/9n/26/fx719n26s`, which `hasModel` = `GwWork`
    - the parent of the proxy record has **2 children**, one of which is the expected `FileSet` record: `http://localhost:8984/rest/prod/g4/45/cd/81/g445cd81f`

In [ ]:
# This should be the usual case
query_normal = '''
PREFIX ns001: <info:fedora/fedora-system:def/model#>
PREFIX ns012: <http://www.openarchives.org/ore/terms/>

SELECT DISTINCT ?s
WHERE 
{
    ?s ns001:hasModel ?o1;
    FILTER (?o1 = "ActiveFedora::Aggregation::Proxy")
    FILTER EXISTS { 
        ?s ns012:proxyFor ?o2.
    }
} 
'''

In [ ]:
norm_results = list(store.query(query_normal))

In [ ]:
len(norm_results)

In [ ]:
norm_results[0]

This record, for instance

- `hasModel` = `ActiveFedora::Aggregation::Proxy`
- `proxyFor` = `http://localhost:8984/rest/prod/s4/65/5g/72/s4655g72c`, which `hasModel` = `FileSet`
- `hasParent` = `http://localhost:8984/rest/prod/hd/76/s0/18/hd76s0189/members`, which `hasModel` = `ActiveFedora::IndirectContainer`
    - the parent `hasParent` = `http://localhost:8984/rest/prod/hd/76/s0/18/hd76s0189`, which `hasModel` = `GwEtd`
    - the parent of the proxy record has only 1 child

In [ ]:
# For those lacking the proxyFor relation, do their parents have VALID children?
query_valid_sibling = '''
PREFIX ns001: <info:fedora/fedora-system:def/model#>
PREFIX ns012: <http://www.openarchives.org/ore/terms/>
PREFIX ldp:   <http://www.w3.org/ns/ldp#>
PREFIX fedora: <http://fedora.info/definitions/v4/repository#>

SELECT DISTINCT ?s ?s1 ?s3
WHERE 
{
    ?s ns001:hasModel ?o1.
    ?s fedora:hasParent ?s1.
    ?s1 ldp:contains ?s3.
    FILTER (?o1 = "ActiveFedora::Aggregation::Proxy")
    FILTER NOT EXISTS { 
        ?s ns012:proxyFor ?o2.
    }
    FILTER EXISTS { 
        ?s3  ns012:proxyFor ?o3.
    }
} 
'''

In [ ]:
siblings = list(store.query(query_valid_sibling))

In [ ]:
len(siblings)

In [ ]:
len({q['s'].value for q in siblings})

In [ ]:
from csv import DictWriter

In [ ]:
fields = ['bad_node', 'parent', 'valid_node']
with open('./nodes_to_test.csv', 'w') as f:
    writer = DictWriter(f, fields)
    writer.writeheader()
    for q in siblings:
        writer.writerow(dict(zip(fields, [s.value for s in q])))

In [ ]:
valid_sibling_ids = {q['s'].value for q in siblings}

In [ ]:
# One Proxy object without a valid sibling
# This work has no FileSet associated with it
{q['s'].value for q in results} - valid_sibling_ids

In [ ]:
# Grandparents of the problematic objects
query_ancestor_work = '''
PREFIX ns001: <info:fedora/fedora-system:def/model#>
PREFIX ns012: <http://www.openarchives.org/ore/terms/>
PREFIX ldp:   <http://www.w3.org/ns/ldp#>
PREFIX fedora: <http://fedora.info/definitions/v4/repository#>
PREFIX dcterms: <http://purl.org/dc/terms/>

SELECT DISTINCT ?work ?title ?date
WHERE 
{
    ?s ns001:hasModel ?o1.
    ?s fedora:hasParent ?s1.
    ?s1 fedora:hasParent ?work.
    ?work dcterms:title ?title.
    ?work fedora:created ?date.
    FILTER (?o1 = "ActiveFedora::Aggregation::Proxy")
    FILTER NOT EXISTS { 
        ?s ns012:proxyFor ?o2.
    }
} 
'''

In [ ]:
works = list(store.query(query_ancestor_work))

In [ ]:
works[0]

In [ ]:
fields = ['uri', 'title', 'date_created']
with open('./works_with_dangling_proxies.csv', 'w') as f:
    writer = DictWriter(f, fields)
    writer.writeheader()
    for q in works:
        writer.writerow(dict(zip(fields, [s.value for s in q])))

In [ ]:
# How many objects point to each of these invalid objects?
query_points_to = '''
PREFIX ns001: <info:fedora/fedora-system:def/model#>
PREFIX ns012: <http://www.openarchives.org/ore/terms/>

SELECT DISTINCT ?s (COUNT(?other) AS ?other_count)
WHERE 
{
    ?s ns001:hasModel ?o1.
    ?other ?p ?s.
    FILTER (?o1 = "ActiveFedora::Aggregation::Proxy")
    FILTER NOT EXISTS { 
        ?s ns012:proxyFor ?o2.
    }
} 
GROUP BY ?s
'''

In [ ]:
points_to = list(store.query(query_points_to))

In [ ]:
[p for p in points_to if int(p['other_count'].value) > 1]

##### Mitigation

1. `DELETE` query for `AggregationProxy` object.
    - Response should be `204`
2. `PATCH` query for `IndirectContainer` object.
```
PREFIX ldp:  <http://www.w3.org/ns/ldp#>
DELETE {
    <> ldp:contains <deleted-uri> .
}
WHERE { }
```

**Note**: It does not appear to be necessary to update the parent of the deleted object; the reference seems to be removed by Fedora upon the object's deletion.

In [ ]:
import requests
from requests import HTTPError

In [ ]:
def delete_object(session, uri):
    try:
        r = session.delete(uri)
        if r.status_code != 204:
            r.raise_for_status()
    except HTTPError:
        print(f'Error deleting {uri}: {r.text}')

In [ ]:
def delete_from_parent(session, uri):
    update_q = '''
    PREFIX ldp:  <http://www.w3.org/ns/ldp#>

    DELETE {{
        <> ldp:contains <{deleted_uri}> .
    }}
    WHERE {{ }}
    '''
    headers = {'Content-Type': 'application/sparql-update'}
    parent_uri = '/'.join(uri.split('/')[:-1])
    try:
        r = session.patch(parent_uri, data=update_q.format(deleted_uri=uri))
        if r.status_code != 204:
            r.raise_for_status()
    except HTTPError:
        print(f'Error patching {uri}: {r.text}')

In [ ]:
test, rest = results[0], results[1:]

In [ ]:
sess = requests.Session()

In [ ]:
delete_object(sess, test['s'].value)

In [ ]:
delete_from_parent(sess, test['s'].value)

In [ ]:
for r in rest:
    delete_object(sess, r['s'].value)

In [ ]:
[r['s'].value for r in rest]

#### Objects outside repo root

In [ ]:
# These seem to be content-admin ACL's, unconnected to other objects
query_ca = '''
PREFIX acl: <http://www.w3.org/ns/auth/acl#>

SELECT (COUNT(?s) as ?count)

WHERE { 
    ?s acl:agent <http://projecthydra.org/ns/auth/group#content-admin>
    
}
'''

In [ ]:
list(store.query(query_ca))

In [ ]:
# Model types for objects outside the root
query_orphans1 = '''
PREFIX ns001: <info:fedora/fedora-system:def/model#>
PREFIX fedora: <http://fedora.info/definitions/v4/repository#>
PREFIX acl: <http://www.w3.org/ns/auth/acl#>

SELECT DISTINCT ?o ?o2 (COUNT(?s) AS ?count) 

WHERE { 
    ?s ns001:hasModel ?o.
    ?s fedora:hasParent <http://localhost:8984/rest/>.
    ?s acl:agent ?o2
}
GROUP BY ?o ?o2
'''

In [ ]:
# All orphans seems to be content-admin ACL's
list(store.query(query_orphans1))

In [ ]:
# Comparing these ACL's to those under the repo root
query_acl = '''
PREFIX ns001: <info:fedora/fedora-system:def/model#>
PREFIX fedora: <http://fedora.info/definitions/v4/repository#>
PREFIX acl: <http://www.w3.org/ns/auth/acl#>

SELECT ?o (COUNT(?s) AS ?count)

WHERE { 
    ?s ns001:hasModel "Hydra::AccessControls::Permission".
    ?s acl:agent <http://projecthydra.org/ns/auth/group#content-admin>.
    ?s fedora:hasParent ?s2.
    ?s2 ns001:hasModel ?o
}
GROUP BY ?o
'''

In [ ]:
list(store.query(query_acl))

All objects with the `acl:agent` predicate of `content-admin`, with the exception of the those outside the repo root, have a parent of the type `Hydra::AccessControl`. Those outside the root have the `rest` endpoint as their parent, which isn't even a proper container. I think, therefore, that these orphans can be excluded from the export.

#### Deletion of non-prod objects

In [ ]:
rest_uri = 'http://localhost:8984/rest/'
headers =  {'Accept': 'application/x-turtle'}

In [ ]:
import requests
rdf = requests.get(rest_uri, headers=headers)

In [ ]:
print(rdf.text)

In [ ]:
prefixes = [r for r in rdf.text.split('\n') if r.startswith('@prefix')]

In [ ]:
prefix_dict = {}
for prefix in prefixes:
    p = prefix.split()
    prefix_dict[p[1]] = p[2][1:-1]

In [ ]:
prefix_dict

In [ ]:
from pyoxigraph import parse, RdfFormat, NamedNode
g = parse(rdf.text, format=RdfFormat.TURTLE)

In [ ]:
print(serialize(input=g, format=RdfFormat.TURTLE, prefixes=prefix_dict).decode())

In [ ]:
non_prod = [node.object.value for node in g if node.predicate == NamedNode('http://www.w3.org/ns/ldp#contains') 
            and node.object != NamedNode('http://localhost:8984/rest/prod')]

In [ ]:
session = requests.session()
for uri in non_prod:
    delete_object(session, uri)

#### Reindexing Error

In [ ]:
list(store.quads_for_pattern(NamedNode('http://localhost:8984/rest/prod/5t/34/sk/59/5t34sk59v'), None, None))

#### Fedora 6 objects

In [ ]:
store_6 = Store.read_only('./db-ocfl')

In [ ]:
list(store_6.quads_for_pattern(NamedNode('info:fedora/prod/xd/07/gt/33/xd07gt33n/members/74a1b297-89a2-4c73-873a-2e6fb86c739c'), None, None))